# Simulate spending

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from src.utils.modelling import *

In [ ]:
import lightgbm as lgb
lgb.Booster(model_file='./outputs/models/lgb.txt')